In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import time
import evaluate
import pandas as pd
import numpy as np

/Users/vmishra/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [15]:
# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
raw_datasets = load_dataset(data_name, cache_dir="../../datasets", split="train")
raw_datasets = raw_datasets.train_test_split(test_size=0.2, shuffle=True)
print(raw_datasets)
raw_datasets["train"][0]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})


{'text': '<s>[INST] Convierte el siguiente texto en tercera persona:\n"Soy un investigador con una sólida formación en Ingeniería Electrónica obtenida en la UTN. Tengo experiencia en aplicaciones ópticas, adquirida durante mi tiempo como becario en el Laboratorio de Holografía, y he participado en diversos proyectos de investigación y desarrollo." [/INST] Él es un investigador con una sólida formación en Ingeniería Electrónica obtenida en la UTN. Tiene experiencia en aplicaciones ópticas, adquirida durante su tiempo como becario en el Laboratorio de Holografía, y ha participado en diversos proyectos de investigación y desarrollo. </s>'}

In [3]:
# Model and tokenizer names
base_model_name = "gpt2"
refined_model = "gpt2-enhanced"

import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

device_type = "mps"
device = torch.device(device_type)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, cache_dir="../../datasets")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

In [4]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 800
})

In [6]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

In [7]:
from peft import LoraConfig, PeftModel
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

In [13]:
tokenized_datasets["train"][0]

{'text': '<s>[INST] ¿En qué materiales se puede imprimir en 3D? [/INST] La impresión 3D utiliza una variedad de materiales para crear objetos tridimensionales. A continuación se presentan algunos de los materiales más comunes utilizados en la impresión 3D:\n\nPlásticos: el plástico es uno de los materiales más utilizados en la impresión 3D. Algunos de los plásticos más comunes incluyen ABS, PLA, PETG, Nylon, TPU, entre otros.\n\nResinas: la resina es un material líquido que se endurece cuando se expone a la luz ultravioleta. Este tipo de material se utiliza en impresoras 3D de resina y es común en la producción de joyas, moldes, y piezas detalladas.\n\nMetales: la impresión 3D de metales se ha vuelto más común en los últimos años y se utiliza en la producción de piezas aeroespaciales, automotrices, y médicas. Algunos de los metales comunes utilizados en la impresión 3D son aluminio, titanio, acero inoxidable y cobre.\n\nCerámica: la impresión 3D de cerámica se utiliza en la producción 

In [9]:
output_dir = f'./peft-qa-training-{str(int(time.time()))}'

# Training Params
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1    
)
    
fine_tuning = Trainer(
    model=base_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [10]:
# Training
fine_tuning.train()
# Save Model
fine_tuning.model.save_pretrained(refined_model)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.